In [3]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sys

sys.path.append('D:\substorm-detection\RNN')
sys.path.append('D:\substorm-detection')
sys.path.append('D:\substorm-detection\\baseline')
sys.path.append('D:\substorm-detection\\data')

import rnn_models
import utils
import utils_linear
import plot_utils
import linear_models

%load_ext autoreload
%autoreload 2

sns.set()

Using TensorFlow backend.


## Load in Data

In [4]:
data_fn = 'D:\\substorm-detection\\data\\short_data.npz'
data = np.load(data_fn)
X = data['X']
y = data['y'][:, None]
SW = data['SW']
strength = data['strength']

In [6]:
print(np.shape(X))
print(np.shape(y))
print(np.shape(SW))
print(np.shape(strength))

(1422, 52, 128, 3)
(1422, 1)
(1422, 256, 6)
(1422,)


In [43]:
params = {
    'batch_size': 5,
    'rnn_hidden_units': 128,
    'n_stacks': 2,
    'fc_hidden_size': 128,
    'n_classes': 3,
    'epochs': 3,
    'verbose': True,
    'time_output_weight': 1
}

In [44]:
train_test_split = .11
train_val_split = .15
train, test = utils.split_data([X, SW, y, strength], train_test_split, random=False)
train, val = utils.split_data(train, train_val_split, random=True)
X_train, sw_train, y_train, strength_train = train
X_val, sw_val, y_val, strength_val = val
X_test, sw_test, y_test, strength_test = test

X_train, X_val, X_test = utils.rnn_format_x([X_train, X_val, X_test])

X_train, X_val, X_test = [X_train, sw_train], [X_val, sw_val], [X_test, sw_test]
y_train, y_val, y_test = [y_train, strength_train], [y_val, strength_val], [y_test, strength_test]

In [47]:
y_test

[array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
      

In [46]:
hist, model = rnn_models.train_functional_gru(X_train, y_train, X_val, y_val, params)

(1075, 128, 156)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (5, 128, 156)        0                                            
__________________________________________________________________________________________________
gru_37 (GRU)                    (5, 128, 128)        109440      input_19[0][0]                   
__________________________________________________________________________________________________
gru_39 (GRU)                    (5, 128, 128)        109440      input_19[0][0]                   
__________________________________________________________________________________________________
gru_38 (GRU)                    (5, 128, 128)        98688       gru_37[0][0]                     
____________________________________________________________________________________________